# A2 INE5405 - Probabilidade e Estatística
## Teste de Hipóteses com Dados da COVID-19 no Brasil: 2020 vs. 2024
- (a) Escolher um tema de sua área, ou de sua preferência (é recomendado manter o tema utilizado
no trabalho 1 para facilitar o desenvolvimento das atividades).
- (b) Selecionar, no mínimo, quatro variáveis de interesse, seguindo as mesmas recomendações do
trabalho 1, sendo possível manter as variáveis já apresentadas.
- (c) Fazer uma breve descrição dos dados, calculando estatísticas descritivas (utilizar estimativas
pontuais e intervalares especialmente para média e proporção, quando for o caso).
- (d) Levantar pelo menos duas hipóteses envolvendo os dados em análise.
- (e) Construir os testes de hipóteses e executá-los com base nos dados, discutindo os resultados.
- (f) Apresentar as conclusões com base nas estatísticas descritivas e testes estatísticos realizados

In [1]:
# Importar bibliotecas necessárias
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sidetable as stb

# Configurar pandas
pd.set_option('display.max_rows', 100)

In [4]:
# Carregar dados do exercício anterior 
df = pd.read_parquet('data/a1_dados_covid.parquet')
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 2996660 entries, 4551 to 427043
Data columns (total 10 columns):
 #   Column             Dtype         
---  ------             -----         
 0   data               datetime64[ns]
 1   regiao             object        
 2   estado             object        
 3   municipio          object        
 4   populacaoTCU2019   float64       
 5   obitosAcumulado    int64         
 6   obitosNovos        int64         
 7   ano                int32         
 8   mes                object        
 9   percentual_obitos  float64       
dtypes: datetime64[ns](1), float64(2), int32(1), int64(2), object(4)
memory usage: 240.1+ MB


## Variáveis escolhidas:
- data (para ordenação)
- regiao
- estado
- municipio 
- populacaoTCU
- obitosAcumulado
- obitosNovos
- ano (para comparação)
- percentual_obitos

## Estatísticas descritivas

In [5]:
df.stb.missing(style=True)

,missing,total,percent
data,0,"2,996,660",0.00%
regiao,0,"2,996,660",0.00%
estado,0,"2,996,660",0.00%
municipio,0,"2,996,660",0.00%
populacaoTCU2019,0,"2,996,660",0.00%
obitosAcumulado,0,"2,996,660",0.00%
obitosNovos,0,"2,996,660",0.00%
ano,0,"2,996,660",0.00%
mes,0,"2,996,660",0.00%
percentual_obitos,0,"2,996,660",0.00%


### tratamento de distância interquartílica para gráficos

In [21]:
q1 = df['obitosAcumulado'].quantile(0.25)
q3 = df['obitosAcumulado'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5*iqr
upper_bound = q3 + 1.5*iqr
df_plot_acc = df[(df['obitosAcumulado'] > lower_bound) & (df['obitosAcumulado'] < upper_bound)]


In [26]:
df[['obitosAcumulado','obitosNovos','percentual_obitos']].describe()

,obitosAcumulado,obitosNovos,percentual_obitos
count,2.996660e+06,2.996660e+06,2.996660e+06
mean,7.042710e+01,6.648035e-02,1.430992e-01
std,6.687656e+02,7.655450e+01,3.238256e-01
min,0.000000e+00,-4.567100e+04,0.000000e+00
25%,1.000000e+00,0.000000e+00,7.633711e-03
50%,8.000000e+00,0.000000e+00,7.609696e-02
75%,3.100000e+01,0.000000e+00,2.393322e-01
max,4.602400e+04,4.567100e+04,1.283539e+02


In [27]:
df[df['ano'] == 2020][['obitosAcumulado','obitosNovos','percentual_obitos']].describe()

,obitosAcumulado,obitosNovos,percentual_obitos
count,1.559600e+06,1.559600e+06,1.559600e+06
mean,1.776725e+01,1.248089e-01,2.522536e-02
std,2.168569e+02,7.231061e+01,3.967055e-02
min,0.000000e+00,-4.095300e+04,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.000000e+00,0.000000e+00,9.652510e-03
75%,6.000000e+00,0.000000e+00,3.887475e-02
max,4.585400e+04,4.118000e+04,1.508496e+01


In [ ]:
df[df['ano'] == 2024][['obitosAcumulado','obitosNovos','percentual_obitos']].describe()

,obitosAcumulado,obitosNovos
count,1.437060e+06,1.437060e+06
mean,1.275773e+02,3.178016e-03
std,9.355853e+02,8.090876e+01
min,0.000000e+00,-4.567100e+04
25%,1.200000e+01,0.000000e+00
50%,2.600000e+01,0.000000e+00
75%,6.400000e+01,0.000000e+00
max,4.602400e+04,4.567100e+04


In [28]:
# Agrupar por municipio e selecionar os top 10
top_mun_2020 = df[df['ano'] == 2020].groupby('municipio').agg({'percentual_obitos':'sum'}).nlargest(10,'percentual_obitos').reset_index()
top_mun_2020

,municipio,percentual_obitos
0,Areia Branca,55.027072
1,Belém,53.057458
2,São Francisco,50.038369
3,Charrua,49.862763
4,Itambé,49.186588
5,Água Branca,48.132930
6,Gastão Vidigal,48.128120
7,Sapucaia,41.578290
8,Pimenteiras do Oeste,40.986630
9,Presidente Kennedy,38.541767


In [29]:
# óbitos por municipio em 2024
# Agrupar por municipio e selecionar os top 10
top_mun_2024 = df[df['data'] == '2024-09-14'].groupby('municipio').agg({'percentual_obitos':'max'}).nlargest(10,'percentual_obitos').reset_index()
top_mun_2024

,municipio,percentual_obitos
0,Santa Rita d'Oeste,2.001601
1,Estrela do Norte,1.482285
2,Santa Clara d'Oeste,1.229314
3,Embu-Guaçu,1.116956
4,Meridiano,1.094891
5,Parisi,0.925497
6,Campos Verdes,0.887436
7,Pirapó,0.868056
8,Itaúba,0.867964
9,Macedônia,0.865333


### Tabelas de frequência

In [30]:
df.stb.freq(['regiao'], style=True)

,regiao,count,percent,cumulative_count,cumulative_percent
0,Nordeste,"965,172",32.21%,"965,172",32.21%
1,Sudeste,"897,384",29.95%,"1,862,556",62.15%
2,Sul,"640,758",21.38%,"2,503,314",83.54%
3,Centro-Oeste,"251,246",8.38%,"2,754,560",91.92%
4,Norte,"242,100",8.08%,"2,996,660",100.00%


In [31]:
df.stb.freq(['estado'], style=True)

,estado,count,percent,cumulative_count,cumulative_percent
0,MG,"458,914",15.31%,"458,914",15.31%
1,SP,"347,010",11.58%,"805,924",26.89%
2,RS,"267,386",8.92%,"1,073,310",35.82%
3,BA,"224,346",7.49%,"1,297,656",43.30%
4,PR,"214,662",7.16%,"1,512,318",50.47%
5,SC,"158,710",5.30%,"1,671,028",55.76%
6,GO,"132,348",4.42%,"1,803,376",60.18%
7,PI,"120,512",4.02%,"1,923,888",64.20%
8,PB,"119,974",4.00%,"2,043,862",68.20%
9,MA,"116,746",3.90%,"2,160,608",72.10%


In [32]:
# Tabela de frequência para a variável 'regiao' e total de óbitos
df[df['data'] == '2024-09-14'].stb.freq(['regiao'],value='obitosAcumulado',style=True)

,regiao,obitosAcumulado,percent,cumulative_obitosAcumulado,cumulative_percent
0,Sudeste,"343,673",48.25%,"343,673",48.25%
1,Nordeste,"136,564",19.17%,"480,237",67.42%
2,Sul,"112,836",15.84%,"593,073",83.26%
3,Centro-Oeste,"67,148",9.43%,"660,221",92.69%
4,Norte,"52,066",7.31%,"712,287",100.00%


In [33]:
df[df['data'] == '2024-09-14'].obitosAcumulado.sum()

np.int64(712287)

In [34]:
# Obitos em 2020 por regiao
df[df['ano'] == 2020].stb.freq(['regiao'], value='obitosNovos',style=True)

,regiao,obitosNovos,percent,cumulative_obitosNovos,cumulative_percent
0,Sudeste,"89,072",45.76%,"89,072",45.76%
1,Nordeste,"47,676",24.49%,"136,748",70.25%
2,Sul,"22,037",11.32%,"158,785",81.57%
3,Norte,"18,019",9.26%,"176,804",90.83%
4,Centro-Oeste,"17,848",9.17%,"194,652",100.00%


In [35]:
# Obitos em 2024 por regiao
df[df['ano'] == 2024].stb.freq(['regiao'], value='obitosNovos',style=True)

,regiao,obitosNovos,percent,cumulative_obitosNovos,cumulative_percent
0,Sudeste,"2,689",58.88%,"2,689",58.88%
1,Sul,688,15.06%,"3,377",73.94%
2,Nordeste,498,10.90%,"3,875",84.85%
3,Centro-Oeste,428,9.37%,"4,303",94.22%
4,Norte,264,5.78%,"4,567",100.00%


In [36]:
df[df['ano'] == 2020].obitosNovos.sum()

np.int64(194652)

In [37]:
df[df['ano'] == 2024].obitosNovos.sum()

np.int64(4567)

In [38]:
# Total de obitos por estado
df[df['data'] == '2024-09-14'].stb.freq(['estado'],value='obitosAcumulado',style=True)

,estado,obitosAcumulado,percent,cumulative_obitosAcumulado,cumulative_percent
0,SP,"183,849",25.81%,"183,849",25.81%
1,RJ,"78,198",10.98%,"262,047",36.79%
2,MG,"66,698",9.36%,"328,745",46.15%
3,PR,"46,710",6.56%,"375,455",52.71%
4,RS,"43,007",6.04%,"418,462",58.75%
5,BA,"31,656",4.44%,"450,118",63.19%
6,GO,"28,619",4.02%,"478,737",67.21%
7,CE,"28,215",3.96%,"506,952",71.17%
8,PE,"23,240",3.26%,"530,192",74.44%
9,SC,"23,119",3.25%,"553,311",77.68%


In [39]:
# Total de obitos por estado em 2020
df[df['ano'] == 2020].stb.freq(['estado'],value='obitosNovos',style=True)

,estado,obitosNovos,percent,cumulative_obitosNovos,cumulative_percent
0,SP,"46,717",24.00%,"46,717",24.00%
1,RJ,"25,530",13.12%,"72,247",37.12%
2,MG,"11,804",6.06%,"84,051",43.18%
3,CE,"9,990",5.13%,"94,041",48.31%
4,PE,"9,654",4.96%,"103,695",53.27%
5,BA,"9,065",4.66%,"112,760",57.93%
6,RS,"8,872",4.56%,"121,632",62.49%
7,PR,"7,912",4.06%,"129,544",66.55%
8,PA,"7,188",3.69%,"136,732",70.24%
9,GO,"6,805",3.50%,"143,537",73.74%


In [40]:
# Total de obitos por estado em 2024
df[df['ano'] == 2024].stb.freq(['estado'],value='obitosNovos',style=True)

,estado,obitosNovos,percent,cumulative_obitosNovos,cumulative_percent
0,SP,"1,595",34.92%,"1,595",34.92%
1,MG,602,13.18%,"2,197",48.11%
2,RJ,433,9.48%,"2,630",57.59%
3,RS,361,7.90%,"2,991",65.49%
4,PR,201,4.40%,"3,192",69.89%
5,GO,172,3.77%,"3,364",73.66%
6,BA,160,3.50%,"3,524",77.16%
7,SC,126,2.76%,"3,650",79.92%
8,PA,123,2.69%,"3,773",82.61%
9,MS,102,2.23%,"3,875",84.85%


In [41]:
# Total de obitos por municipio (top 10)
df[df['data'] == '2024-09-14'].stb.freq(['municipio'],value='obitosAcumulado',thresh = 22,style=True)

,municipio,obitosAcumulado,percent,cumulative_obitosAcumulado,cumulative_percent
0,São Paulo,"46,024",6.46%,"46,024",6.46%
1,Rio de Janeiro,"38,661",5.43%,"84,685",11.89%
2,Brasília,"12,014",1.69%,"96,699",13.58%
3,Fortaleza,"11,818",1.66%,"108,517",15.24%
4,Manaus,"9,966",1.40%,"118,483",16.63%
5,Salvador,"9,222",1.29%,"127,705",17.93%
6,Curitiba,"8,929",1.25%,"136,634",19.18%
7,Belo Horizonte,"8,671",1.22%,"145,305",20.40%
8,Goiânia,"8,167",1.15%,"153,472",21.55%
9,others,"558,815",78.45%,"712,287",100.00%


## **Com base nas informações obtidas, formular hipóteses:**

<img src='https://caelum-online-public.s3.amazonaws.com/1229-estatistica-parte3/01/img003.png' width=70%>

> ![Níveis de Confiança e significância](https://caelum-online-public.s3.amazonaws.com/1229-estatistica-parte3/01/img001.png)

<img src='https://caelum-online-public.s3.amazonaws.com/1229-estatistica-parte3/01/img013.png' width=90%>

### 1a Hipótese: a média de óbitos na região sudeste é superior à média amostral
#### Existe evidência suficiente no nível α = 0.05 para concluir que a média de óbitos da região sudeste é superior à média de todas as regiões?

### **Passo 1** - formulação das hipóteses $H_0$ e $H_1$

#### <font color='red'>Lembre-se, a hipótese nula sempre contém a alegação de igualdade</font>

### $H_0: \mu = x$

### $H_1: \mu \neq x$

### **Passo 2** - escolha da distribuição amostral adequada

### O tamanho da amostra é maior que 30?
#### Resp.: Sim

### O desvio padrão populacional é conhecido?
#### Resp.: Não

### **Passo 3** - fixação da significância do teste ($\alpha$)

### **Passo 4** - cálculo da estatística-teste e verificação desse valor com as áreas de aceitação e rejeição do teste

# $$z = \frac{\bar{x} - \mu_0}{\frac{s}{\sqrt{n}}}$$

### Critério do $p-valor$ 

> ### Teste Bicaudal
> ### Rejeitar $H_0$ se o valor $p\leq\alpha$

### 2a hipótese: após a vacinação, a média de óbitos da região sudeste diminuiu em pelo menos 40%, considerando o período após a vacinação a partir de 2022.
#### Existe evidência suficiente no nível α = 0.05 para concluir que a média de óbitos na região sudeste a partir de 2022 é pelo menos 40% menor que a média no período de 2020-2022?